In [1]:
import sys
import numpy as np
import torch
import os, glob
import smplx
import json

In [2]:
from GRAB.tools.objectmodel import ObjectModel
from GRAB.tools.utils import parse_npz, prepare_params, params2torch, to_cpu, append2dict
from GRAB.tools.meshviewer import Mesh

from manopth.manolayer import ManoLayer

/home/erik/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:107: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
grab_dir = os.path.abspath('/media/erik/DATA/grab_extracted/grab')
grab_amass_dir = os.path.abspath('/media/erik/DATA/grab_amass')
out_dir = os.path.abspath('/media/erik/DATA/grab_preprocessed')
model_path = os.path.abspath('./body_models/')
text_annotation_path = os.path.abspath('./grab_annotations.csv')
text_annotation_out_dir = os.path.abspath('/media/erik/DATA/texts/')
os.makedirs(out_dir, exist_ok=True)

print('Using device:', device)
print('Data directory:', grab_dir)
print('AMASS directory:', grab_amass_dir)
print('Output directory:', out_dir)
print('SMPLX/MANO Model directory:', model_path)
print('Text annotation file:', text_annotation_path)
print('Text annotation output directory:', text_annotation_out_dir)

Using device: cpu
Data directory: /media/erik/DATA/grab_extracted/grab
AMASS directory: /media/erik/DATA/grab_amass
Output directory: /media/erik/DATA/grab_preprocessed
SMPLX/MANO Model directory: /home/erik/ethz/digital-humans/dex-hoi/data_preprocessing/grab_preprocessing/body_models
Text annotation file: /home/erik/ethz/digital-humans/dex-hoi/data_preprocessing/grab_preprocessing/grab_annotations.csv
Text annotation output directory: /media/erik/DATA/texts


In [4]:
# map directory names to lists of files in them
data_dict = {}

for root, dirs, files in os.walk(grab_dir):
    if len(files) > 0:
        print(f'Adding directory {root} with {len(files)} files, creating output folder structure...')
        subject_id = os.path.basename(root)
        data_dict[subject_id] = {}

        for file in files:
            task_description = os.path.splitext(file)[0]
            file_in = os.path.join(root, file)
            file_out_dir = os.path.join(out_dir, f'{subject_id}_{task_description}')
            os.makedirs(file_out_dir, exist_ok=True)
            data_dict[subject_id][task_description] = {'full_info': file_in, 'preprocessed_out': file_out_dir}

for root, dirs, files in os.walk(grab_amass_dir):
    subject_id = os.path.basename(root)
    if subject_id in data_dict:
        for file in files:
            # all processed files should have 'stageii' in their name
            if 'stageii' not in file:
                continue

            task_description = os.path.splitext(file)[0].replace('_stageii', '')
            if task_description in data_dict[subject_id]:
                data_dict[subject_id][task_description]['amass_info'] = os.path.join(root, file)
            elif 'pick_all' in task_description:
                task_description = task_description.replace('pick_all', 'lift')
                if task_description in data_dict[subject_id]:
                    data_dict[subject_id][task_description]['amass_info'] = os.path.join(root, file)
                else:
                    print(f'No corresponding task for {task_description} in {subject_id} found')
            else:
                print(f'No corresponding task for {task_description} in {subject_id} found')



Adding directory /media/erik/DATA/grab_extracted/grab/s2 with 93 files, creating output folder structure...
Adding directory /media/erik/DATA/grab_extracted/grab/s6 with 148 files, creating output folder structure...
Adding directory /media/erik/DATA/grab_extracted/grab/s9 with 125 files, creating output folder structure...
Adding directory /media/erik/DATA/grab_extracted/grab/s1 with 198 files, creating output folder structure...
Adding directory /media/erik/DATA/grab_extracted/grab/s10 with 145 files, creating output folder structure...
Adding directory /media/erik/DATA/grab_extracted/grab/s5 with 106 files, creating output folder structure...
Adding directory /media/erik/DATA/grab_extracted/grab/s8 with 162 files, creating output folder structure...
Adding directory /media/erik/DATA/grab_extracted/grab/s3 with 125 files, creating output folder structure...
Adding directory /media/erik/DATA/grab_extracted/grab/s4 with 113 files, creating output folder structure...
Adding directory /m

In [5]:
from text_preprocessing import process_grab
import pandas as pd

# load csv dataset as dataframe
df = pd.read_csv(text_annotation_path, header=None, names=['seq_name', 'caption'])
os.makedirs(text_annotation_out_dir, exist_ok=True)

process_grab(df, out_path=text_annotation_out_dir)
    
    


100%|██████████| 1333/1333 [00:06<00:00, 218.89it/s]


In [6]:
def load_sbj_verts(seq_data):
        mesh_path = os.path.join(grab_dir, '..',seq_data.body.vtemp)
        sbj_vtemp = np.array(Mesh(filename=mesh_path).vertices)
        return sbj_vtemp

def load_obj_verts(obj_name, seq_data, n_verts_sample=512):
    mesh_path = os.path.join(grab_dir, '..',seq_data.object.object_mesh)
    np.random.seed(100)
    obj_mesh = Mesh(filename=mesh_path)
    verts_obj = np.array(obj_mesh.vertices)
    faces_obj = np.array(obj_mesh.faces)

    if verts_obj.shape[0] > n_verts_sample:
        verts_sample_id = np.random.choice(verts_obj.shape[0], n_verts_sample, replace=False)
    else:
        verts_sample_id = np.arange(verts_obj.shape[0])

    verts_sampled = verts_obj[verts_sample_id]
    obj_info = {'verts': verts_obj,
                'faces': faces_obj,
                'verts_sample_id': verts_sample_id,
                'verts_sample': verts_sampled,
                'obj_mesh_file': mesh_path}

    return obj_info

save_body_verts = False
save_lhand_verts = True
save_rhand_verts = True
save_object_verts = True
save_contact = True
n_verts_sample = 1000


def process_data_entry(in_file, amass_file, out_dir):
    body_data = {
        'global_orient': [],'body_pose': [],'transl': [],
        'right_hand_pose': [],'left_hand_pose': [],
        'jaw_pose': [],'leye_pose': [],'reye_pose': [],
        'expression': [],'fullpose': [],
        'contact':[], 'verts' :[]
    }

    object_data = {'verts': [], 'global_orient': [], 'transl': [], 'contact': []}
    lhand_data = {'verts': [], 'global_orient': [], 'hand_pose': [], 'transl': [], 'fullpose': [], 'joints': []}
    rhand_data = {'verts': [], 'global_orient': [], 'hand_pose': [], 'transl': [], 'fullpose': [], 'joints': []}

    seq_data = parse_npz(in_file)
    amass_data = np.load(amass_file, allow_pickle=True)
    smplh_data = {
         'poses': amass_data['poses'], # T x 156
         'betas': amass_data['betas'], # 16 -> need to reshape to T x 10
         'trans': amass_data['trans'], # T x 3
        #  'root_orient': amass_data['root_orient'], # T x 3, don't need this for BEHAVE format
    }

    # reshape betas to T x 10 with np.tile
    smplh_data['betas'] = np.tile(smplh_data['betas'][:10], (smplh_data['poses'].shape[0], 1))
    num_amass_timesteps = smplh_data['poses'].shape[0]
    
    obj_name = seq_data.obj_name
    sbj_id   = seq_data.sbj_id
    n_comps  = seq_data.n_comps
    gender   = seq_data.gender

    # need this for other methods from GRAB, this should not filter out any frames
    frame_mask = (seq_data['contact']['object']>-1).any(axis=1)
    T = frame_mask.sum()
    
    # make sure AMASS data has the same number of timesteps as GRAB data, otherwise something is wrong
    assert num_amass_timesteps == T, f'Number of timesteps in GRAB and AMASS data do not match: {num_amass_timesteps} vs {T}'

    sbj_params = prepare_params(seq_data.body.params, frame_mask)
    rh_params  = prepare_params(seq_data.rhand.params, frame_mask)
    lh_params  = prepare_params(seq_data.lhand.params, frame_mask)
    obj_params = prepare_params(seq_data.object.params, frame_mask)

    append2dict(body_data, sbj_params)
    append2dict(rhand_data, rh_params)
    append2dict(lhand_data, lh_params)
    append2dict(object_data, obj_params)

    sbj_vtemp = load_sbj_verts(seq_data)

    if save_body_verts:

        sbj_m = smplx.create(model_path=model_path,
                                model_type='smplx',
                                gender=gender,
                                num_pca_comps=n_comps,
                                v_template=sbj_vtemp,
                                batch_size=T)

        sbj_parms = params2torch(sbj_params)
        verts_sbj = to_cpu(sbj_m(**sbj_parms).vertices)
        body_data['verts'].append(verts_sbj)

    if save_lhand_verts:
        lh_mesh = os.path.join(grab_dir, '..', seq_data.lhand.vtemp)
        lh_vtemp = np.array(Mesh(filename=lh_mesh).vertices)

        lh_m = smplx.create(model_path=model_path,
                            model_type='mano',
                            is_rhand=False,
                            v_template=lh_vtemp,
                            num_pca_comps=n_comps,
                            flat_hand_mean=True,
                            batch_size=T)

        lh_parms = params2torch(lh_params)
        model_output = lh_m(**lh_parms)
        verts_lh = to_cpu(model_output.vertices)

        hand_pose = model_output.hand_pose # T x 45-dim
        betas = model_output.betas # T x 10-dim
        joints = model_output.joints # T x 16 x 3

        mano_layer = ManoLayer(
            side='right',
            mano_root=f'{model_path}/mano/',
            use_pca=False,
            ncomps=45,
            flat_hand_mean=True,
            center_idx=0
        ).to(device)

        hand_pose = hand_pose.to(device)
        betas = betas.to(device)
        translation = torch.from_numpy(rh_params['transl']).to(device)
        full_hand_pose = torch.cat([hand_pose, torch.from_numpy(rh_params['global_orient']).to(device)], dim=1)

        _, hand_joints = mano_layer(full_hand_pose, betas, translation)
        hand_joints = to_cpu(hand_joints)

        lhand_data['joints'].append(hand_joints)

        # lhand_data['verts'].append(verts_lh)

    if save_rhand_verts:
        rh_mesh = os.path.join(grab_dir, '..', seq_data.rhand.vtemp)
        rh_vtemp = np.array(Mesh(filename=rh_mesh).vertices)

        rh_m = smplx.create(model_path=model_path,
                            model_type='mano',
                            is_rhand=True,
                            v_template=rh_vtemp,
                            num_pca_comps=n_comps,
                            flat_hand_mean=True,
                            batch_size=T)

        rh_parms = params2torch(rh_params)
        model_output = rh_m(**rh_parms)
        verts_rh = to_cpu(model_output.vertices)
        hand_pose = model_output.hand_pose # T x 45-dim
        betas = model_output.betas # T x 10-dim
        joints = model_output.joints # T x 16 x 3

        mano_layer = ManoLayer(
            side='right',
            mano_root=f'{model_path}/mano/',
            use_pca=False,
            ncomps=45,
            flat_hand_mean=True,
            center_idx=0
        ).to(device)

        hand_pose = hand_pose.to(device)
        betas = betas.to(device)
        translation = torch.from_numpy(rh_params['transl']).to(device)
        full_hand_pose = torch.cat([hand_pose, torch.from_numpy(rh_params['global_orient']).to(device)], dim=1)

        _, hand_joints = mano_layer(full_hand_pose, betas, translation)
        hand_joints = to_cpu(hand_joints)

        rhand_data['joints'].append(hand_joints)
        # rhand_data['verts'].append(verts_rh)

    ### for objects

    obj_info = load_obj_verts(obj_name, seq_data, n_verts_sample)

    if save_object_verts:

        obj_m = ObjectModel(v_template=obj_info['verts_sample'],
                            batch_size=T)
        obj_parms = params2torch(obj_params)
        verts_obj = to_cpu(obj_m(**obj_parms).vertices)
        object_data['verts'].append(verts_obj)

    if save_contact:
        body_data['contact'].append(seq_data.contact.body[frame_mask])
        object_data['contact'].append(seq_data.contact.object[frame_mask][:,obj_info['verts_sample_id']])

    
    behave_format_object_data = {
        'angles': object_data['global_orient'][0], # T x 3
        'trans': object_data['transl'][0] # T x 3
    }

    out_data = [body_data, lhand_data, rhand_data, object_data, smplh_data, behave_format_object_data]
    out_data_name = ['body_data', 'rhand_data', 'lhand_data', 'object_data', 'smpl_fit_all', 'object_fit_all']
    # save with numpy npz
    for i in range(len(out_data)):
        out_file = os.path.join(out_dir, f'{out_data_name[i]}.npz')
        np.savez_compressed(out_file, **out_data[i])

    info_dict = {
        'gender': gender,
    }
    json_str = json.dumps(info_dict)
    with open(os.path.join(out_dir, 'info.json'), 'w') as f:
        f.write(json_str)

    print(f'Processed {in_file} and saved to {out_dir}')


In [7]:
from tqdm.auto import tqdm
# iterate over all files and process them, show tqdm progress bar
for subject_id, task_description in data_dict.items():
    for task_dict in tqdm(task_description.values()):
        try:
            in_file = task_dict['full_info']
            amass_file = task_dict['amass_info']
            out_dir = task_dict['preprocessed_out']
        except KeyError:
            print(f'No file found for {subject_id} - {task_description}')
            continue
        process_data_entry(in_file, amass_file, out_dir)

  0%|          | 0/93 [00:00<?, ?it/s]

/home/erik/ethz/digital-humans/dex-hoi/common/manopth/manopth/manolayer.py:72: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:206.)
  torch.Tensor(smpl_data['betas'].r).unsqueeze(0))


Processed /media/erik/DATA/grab_extracted/grab/s2/flashlight_on_1.npz and saved to /media/erik/DATA/grab_preprocessed/s2_flashlight_on_1
Processed /media/erik/DATA/grab_extracted/grab/s2/toothbrush_lift.npz and saved to /media/erik/DATA/grab_preprocessed/s2_toothbrush_lift
Processed /media/erik/DATA/grab_extracted/grab/s2/apple_lift.npz and saved to /media/erik/DATA/grab_preprocessed/s2_apple_lift
Processed /media/erik/DATA/grab_extracted/grab/s2/stanfordbunny_inspect_1.npz and saved to /media/erik/DATA/grab_preprocessed/s2_stanfordbunny_inspect_1
Processed /media/erik/DATA/grab_extracted/grab/s2/torussmall_lift.npz and saved to /media/erik/DATA/grab_preprocessed/s2_torussmall_lift
Processed /media/erik/DATA/grab_extracted/grab/s2/mug_pass_1.npz and saved to /media/erik/DATA/grab_preprocessed/s2_mug_pass_1
Processed /media/erik/DATA/grab_extracted/grab/s2/doorknob_use_2.npz and saved to /media/erik/DATA/grab_preprocessed/s2_doorknob_use_2
Processed /media/erik/DATA/grab_extracted/grab/

  0%|          | 0/148 [00:00<?, ?it/s]

Processed /media/erik/DATA/grab_extracted/grab/s6/flashlight_on_1.npz and saved to /media/erik/DATA/grab_preprocessed/s6_flashlight_on_1
Processed /media/erik/DATA/grab_extracted/grab/s6/watch_set_2.npz and saved to /media/erik/DATA/grab_preprocessed/s6_watch_set_2
Processed /media/erik/DATA/grab_extracted/grab/s6/torusmedium_pass_1.npz and saved to /media/erik/DATA/grab_preprocessed/s6_torusmedium_pass_1
Processed /media/erik/DATA/grab_extracted/grab/s6/spheresmall_lift.npz and saved to /media/erik/DATA/grab_preprocessed/s6_spheresmall_lift
Processed /media/erik/DATA/grab_extracted/grab/s6/doorknob_use_1.npz and saved to /media/erik/DATA/grab_preprocessed/s6_doorknob_use_1
Processed /media/erik/DATA/grab_extracted/grab/s6/apple_lift.npz and saved to /media/erik/DATA/grab_preprocessed/s6_apple_lift
Processed /media/erik/DATA/grab_extracted/grab/s6/cup_drink_2.npz and saved to /media/erik/DATA/grab_preprocessed/s6_cup_drink_2
Processed /media/erik/DATA/grab_extracted/grab/s6/stanfordbun

...

Processed /media/erik/DATA/grab_extracted/grab/s3/flashlight_on_1.npz and saved to /media/erik/DATA/grab_preprocessed/s3_flashlight_on_1
Processed /media/erik/DATA/grab_extracted/grab/s3/camera_offhand_1.npz and saved to /media/erik/DATA/grab_preprocessed/s3_camera_offhand_1
Processed /media/erik/DATA/grab_extracted/grab/s3/torusmedium_pass_1.npz and saved to /media/erik/DATA/grab_preprocessed/s3_torusmedium_pass_1
Processed /media/erik/DATA/grab_extracted/grab/s3/doorknob_use_1.npz and saved to /media/erik/DATA/grab_preprocessed/s3_doorknob_use_1
Processed /media/erik/DATA/grab_extracted/grab/s3/apple_lift.npz and saved to /media/erik/DATA/grab_preprocessed/s3_apple_lift
Processed /media/erik/DATA/grab_extracted/grab/s3/cup_drink_2.npz and saved to /media/erik/DATA/grab_preprocessed/s3_cup_drink_2
Processed /media/erik/DATA/grab_extracted/grab/s3/stanfordbunny_inspect_1.npz and saved to /media/erik/DATA/grab_preprocessed/s3_stanfordbunny_inspect_1
Processed /media/erik/DATA/grab_extra

  0%|          | 0/113 [00:00<?, ?it/s]

Processed /media/erik/DATA/grab_extracted/grab/s4/watch_set_2.npz and saved to /media/erik/DATA/grab_preprocessed/s4_watch_set_2
Processed /media/erik/DATA/grab_extracted/grab/s4/spheresmall_lift.npz and saved to /media/erik/DATA/grab_preprocessed/s4_spheresmall_lift
Processed /media/erik/DATA/grab_extracted/grab/s4/doorknob_use_1.npz and saved to /media/erik/DATA/grab_preprocessed/s4_doorknob_use_1
Processed /media/erik/DATA/grab_extracted/grab/s4/apple_lift.npz and saved to /media/erik/DATA/grab_preprocessed/s4_apple_lift
Processed /media/erik/DATA/grab_extracted/grab/s4/cup_drink_2.npz and saved to /media/erik/DATA/grab_preprocessed/s4_cup_drink_2
Processed /media/erik/DATA/grab_extracted/grab/s4/stanfordbunny_inspect_1.npz and saved to /media/erik/DATA/grab_preprocessed/s4_stanfordbunny_inspect_1
Processed /media/erik/DATA/grab_extracted/grab/s4/torussmall_lift.npz and saved to /media/erik/DATA/grab_preprocessed/s4_torussmall_lift
Processed /media/erik/DATA/grab_extracted/grab/s4/c

  0%|          | 0/120 [00:00<?, ?it/s]

Processed /media/erik/DATA/grab_extracted/grab/s7/knife_chop_1.npz and saved to /media/erik/DATA/grab_preprocessed/s7_knife_chop_1
Processed /media/erik/DATA/grab_extracted/grab/s7/flashlight_on_1.npz and saved to /media/erik/DATA/grab_preprocessed/s7_flashlight_on_1
Processed /media/erik/DATA/grab_extracted/grab/s7/doorknob_use_1.npz and saved to /media/erik/DATA/grab_preprocessed/s7_doorknob_use_1
Processed /media/erik/DATA/grab_extracted/grab/s7/apple_lift.npz and saved to /media/erik/DATA/grab_preprocessed/s7_apple_lift
Processed /media/erik/DATA/grab_extracted/grab/s7/stanfordbunny_inspect_1.npz and saved to /media/erik/DATA/grab_preprocessed/s7_stanfordbunny_inspect_1
Processed /media/erik/DATA/grab_extracted/grab/s7/doorknob_use_2.npz and saved to /media/erik/DATA/grab_preprocessed/s7_doorknob_use_2
Processed /media/erik/DATA/grab_extracted/grab/s7/eyeglasses_clean_1.npz and saved to /media/erik/DATA/grab_preprocessed/s7_eyeglasses_clean_1
Processed /media/erik/DATA/grab_extract